Load in Carbonara Data Tools

In [1]:
import CarbonaraDataTools as CDT
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio
pio.renderers.default='notebook'

* Enter the path to your protein (it should be inside newFitData)
* Enter the name of the run you're interested in

In [2]:
MolPath = "newFitData/LysozymeBin2/"
RunName = "long/"

Run the below cell to view the logFiles from your run

In [3]:
logs = CDT.getLogs(MolPath,RunName)
logs

['newFitData/LysozymeBin2/long/fitLog1.dat',
 'newFitData/LysozymeBin2/long/fitLog2.dat',
 'newFitData/LysozymeBin2/long/fitLog3.dat',
 'newFitData/LysozymeBin2/long/fitLog4.dat',
 'newFitData/LysozymeBin2/long/fitLog5.dat',
 'newFitData/LysozymeBin2/long/fitLog6.dat']

Check the progress of your run with the below

In [4]:
'<--- Press the play button here to view the dropdown box'

logFile_Dropdown = widgets.Dropdown(
    options = logs,
    value = logs[0],
    description = 'Select a logFile from the dropdown box ',
    disabled=False,
    layout={'width': '550px','height':'25px'}
)

# Function to display tail of DataFrame
def display_tail(log_path):
    df = CDT.LogFile2df(log_path)
    display(df.tail())

def dropdown_handler(change):
    global log_path
    log_path = change.new
    with output:
        clear_output(wait=True)  # Clear previous output
        display_tail(log_path)
        CDT.viewBestSAXSFit(MolPath,log_path).show()
        CDT.viewBestMolChange(MolPath,log_path)

# Apply CSS style to the dropdown box to make it more visible
logFile_Dropdown.style = {'description_width': '250px','description_height': '25px'}

# Output widget
output = widgets.Output()

# Display initial output
log_path = logs[0]
with output:
    display_tail(log_path)
    CDT.viewBestSAXSFit(MolPath,log_path).show()
    CDT.viewBestMolChange(MolPath,log_path)

logFile_Dropdown.observe(dropdown_handler,names='value')
display(logFile_Dropdown)
display(output)

Dropdown(description='Select a logFile from the dropdown box ', layout=Layout(height='25px', width='550px'), o…

Output()

* Run the cell below to view the changes to your structure as we consider more of the SAXs data.
* Change qMax with the slider

In [ ]:
'<--- press the play button here to view the slider'
q = CDT.getqChanges(log_path)
qrange_slider = widgets.SelectionSlider(
    options=q,
    value = q[0],
    description='Max q:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '400px', 'height': '100px'}
    )

def slider_handler(change):
    global q_choice
    q_choice = change.new
    with output:
        clear_output(wait=False)
        saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q_choice)
        CDT.plotMolAndSAXS(MolPath,saxs_fl,mol_fl).show("notebook")

display(qrange_slider)
saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q[0])
# Output widget
output = widgets.Output()
with output:
    CDT.plotMolAndSAXS(MolPath,saxs_fl,mol_fl).show("notebook")   
qrange_slider.observe(slider_handler,names='value')
display(output)

In [ ]:
CDT.PlotBestOutputRMSD(MolPath,RunName)

In [ ]:
CDT.cluster(CDT.BestOutputRMSD(MolPath,RunName))

In [ ]:
CDT.plotBestSKMTComp(MolPath,RunName)

In [ ]:
CDT.overlayCluster(MolPath,RunName,0)